In [2]:
import os
import sys
os.chdir("../")
sys.path.append("../")

import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from thop import profile
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.TwoTowerBaseModel import TwoTowerBaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.TESRec import TESRec
from models.XFormer import XFormer
from models.PLM import PLM
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [18]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))
# m4 = AutoModel.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# m5 = AutoModel.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")
# m6 = AutoModel.from_pretrained('distilbert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m7 = AutoModel.from_pretrained("google/bigbird-roberta-base", cache_dir=config.path + "bert_cache/")
m8 = AutoModel.from_pretrained("allenai/longformer-base-4096", cache_dir=config.path + "bert_cache/")

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', cache_dir=config.path + "bert_cache/")
# t4 = AutoTokenizer.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# t5 = AutoTokenizer.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")
# t6 = AutoTokenizer.from_pretrained('distilbert-base-uncased', cache_dir=config.path + "bert_cache/")
# t7 = AutoTokenizer.from_pretrained("google/bigbird-roberta-base", cache_dir=config.path + "bert_cache/")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
m8.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [19]:
m8

LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0): LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (o

In [2]:
config.bert = 'bert'
config.mode = "encode"
config.scale = "large"

manager = Manager(config)

loaders = manager.prepare()
X1 = list(loaders[0])
# X2 = list(loaders[1])
x1 = X1[0]
# x2 = X2[0]

[2021-11-13 03:11:03,192] INFO (utils.Manager) Hyper Parameters are {'scale': 'large', 'mode': 'encode', 'batch_size': 5, 'batch_size_news': 500, 'k': 3, 'threshold': -inf, 'abs_length': 40, 'signal_length': 100, 'news': None, 'case': False, 'his_size': 50, 'cdd_size': 5, 'impr_size': 2000, 'dropout_p': 0.2, 'lr': 0.0001, 'bert_lr': 3e-05, 'embedding': 'bert', 'encoderN': 'cnn', 'encoderU': 'lstm', 'selector': 'sfi', 'reducer': 'matching', 'ranker': 'onepass', 'pooler': 'attn', 'bert_dim': 768, 'embedding_dim': 768, 'hidden_dim': 384, 'base_rank': 0, 'world_size': 0, 'seed': 42, 'granularity': 'token', 'debias': False, 'full_attn': True, 'descend_history': False, 'shuffle_pos': False, 'save_pos': False, 'sep_his': False, 'diversify': False, 'no_dedup': False, 'segment_embed': False, 'no_rm_punc': False, 'fast': False, 'scheduler': 'linear', 'warmup': 100, 'shuffle': False, 'bert': 'bert', 'tb': False}
[2021-11-13 03:11:03,193] INFO (utils.Manager) preparing dataset...


In [8]:
# embedding = BERT_Embedding(manager)
manager.hidden_dim = 768
manager.reducer = "none"
# encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

# reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

# model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)
model = PLM(manager, encoderU).to(manager.device)

In [13]:
# GateFormer
embedding = 50 * 2 * 100 * 768 * 2

ext_encoding = 50*(2*3*150*100 + 4*100*150) + 8*2*150*150*50
reduction = (2*100*150 + 100*math.log2(100))*50 + 50*3

bert_embed = 150 * 2 * 768 * 2
bert_project = 150*12*64*2
bert_attn = 150*12*64*150*2 + 12*150*150*64*2
bert_intm = 150*768*768*2 + 150*768*2 + 150*768*3072*4
bert_pool = 150*768*4

sum = embedding + ext_encoding + reduction + bert_embed + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

19988582169.28095

In [17]:
# BigBird
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 1024*12*64*2
bert_attn = 1024*12*64*2 + 12*1024*64*2
bert_intm = 1024*768*768*2 + 1024*768*2 + 1024*768*3072*4
bert_pool = 1024*768*4

sum = embedding + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

130584354816

In [ ]:
# longformer
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 1024*12*64*2
bert_attn = 1024*512*12*64*2 + 12*1024*512*64*2
bert_intm = 1024*768*768*2 + 1024*768*2 + 1024*768*3072*4
bert_pool = 1024*768*4

sum = embedding + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [6]:
# EBNR
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 50*12*100*64*2*3
bert_attn = 50*12*100*64*100*2 + 50*12*100*100*64*2
bert_intm = 50*100*768*768*2 + 50*100*768*2 + 50*100*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

656387481600

In [7]:
# Synthesizer
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 50*12*100*64*2*3
bert_attn = 50*12*100*64*100*2 + 50*12*100*100*64*2
bert_intm = 50*100*768*768*2 + 50*100*768*2 + 50*100*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

656387481600

In [8]:
# funnel
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 50*12*100*64*2*3
bert_attn = 50*12*100*64*100*2 + 50*12*100*100*64*2
bert_intm = 50*100*768*768*2 + 50*100*768*2 + 50*100*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 10 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

547085721600

In [9]:
# newbert
embedding = 50 * 2 * 100 * 768 * 2 * 3

bert_project = 50*12*100*64*2*3
bert_attn = 50*12*100*64*100*2 + 50*12*100*100*64*2
bert_intm = 50*100*768*768*2 + 50*100*768*2 + 50*100*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 4 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

219180441600

In [10]:
# distilbert
embedding = 50 * 2 * 100 * 768 * 2 * 2

bert_project = 50*12*100*64*2*3
bert_attn = 50*12*100*64*100*2 + 50*12*100*100*64*2
bert_intm = 50*100*768*768*2 + 50*100*768*2 + 50*100*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 5 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

273815961600